# Setup

In [1]:
# To access datasets stored in personal google drive: grant permissions to access drive.

"""
For the local demo, synching with google drive is not needed.
"""
#from google.colab import drive
#drive.mount('/content/drive')

'\nFor the local demo, synching with google drive is not needed.\n'

In [3]:
# In order to open all of the recipe bearing files in the drive:

import glob
from datetime import datetime

recipe_files = glob.glob("recipes" + "/*.mmf")

def get_recipe(filepath):
  file = open(filepath, 'rb')
  return file.read().decode('ISO-8859-1')

def get_recipes(directory):
  text = ""
  for filename in directory:
    file = open(filename, 'rb')
    text += file.read().decode('ISO-8859-1')
  return text.lower()

"""
train_index = int(len(recipe_files) * 0.6)
test_index = int(train_index + len(recipe_files) * 0.2) + 1


train_files = recipe_files[: train_index]
test_files = recipe_files[train_index: test_index]
dev_files = recipe_files[test_index: ]
"""

train_files = test_files =  dev_files = recipe_files


print("train length: ", len(train_files))
print("test length: ", len(test_files))
print("dev length: ", len(dev_files))


train length:  1
test length:  1
dev length:  1


In [4]:
"""
Text data already processed and written back. Unecessary for demo.
"""

"""

RECIPE_SEPERATOR = "[END]"

for filename in recipe_files:
  recipe = get_recipe(filename)
  lines = recipe.splitlines(keepends=True)
  for i, line in enumerate(lines):
    if line.startswith("MMMM"):
      lines[i] = RECIPE_SEPERATOR
  out_file = open(filename, 'w', encoding='utf-8')
  out_file.writelines(lines)
  out_file.close()
"""

'\n\nRECIPE_SEPERATOR = "[END]"\n\nfor filename in recipe_files:\n  recipe = get_recipe(filename)\n  lines = recipe.splitlines(keepends=True)\n  for i, line in enumerate(lines):\n    if line.startswith("MMMM"):\n      lines[i] = RECIPE_SEPERATOR\n  out_file = open(filename, \'w\', encoding=\'utf-8\')\n  out_file.writelines(lines)\n  out_file.close()\n'

# Char-rnn

In [6]:
import pickle


def create_char_mapping():
  chars = set()
  for file in recipe_files:
    recipe = get_recipe(file)
    chars.update(recipe.lower())
  chars = sorted(list(chars))
  char_to_idx = dict((c, i) for i, c in enumerate(chars))
  idx_to_char = dict((i, c) for i, c in enumerate(chars))
  return char_to_idx, idx_to_char


def write_char_vocab_to_file(idx_to_char):
  chars = [idx_to_char[i] for i in range(len(idx_to_char))]
  with open("chars.pkl", "wb") as file:
    pickle.dump(chars, file)


def load_char_mapping():
  with open("chars.pkl", "rb") as file:
    chars = pickle.load(file)

  char_to_idx = dict((c, i) for i, c in enumerate(chars))
  idx_to_char = dict((i, c) for i, c in enumerate(chars))
  return char_to_idx, idx_to_char


# char_to_idx, idx_to_char = create_char_mapping()
# write_char_vocab_to_file(idx_to_char)

char_to_idx, idx_to_char = load_char_mapping()
print(len(char_to_idx), len(idx_to_char))


125 125


In [7]:
CHAR_LSTM_LAYERS = 2
CHAR_LSTM_UNITS = 256
CHAR_SEQUENCE_LEN = 25
CHAR_STEP_SIZE = 5
CHAR_EPOCHS = 100
CHAR_BATCH_SIZE = 256
CHAR_DROPOUT = .1
CHAR_LR = 0.0005

In [8]:
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, LSTM, Embedding, Dropout, CuDNNLSTM
from tensorflow.python.keras.callbacks import ModelCheckpoint
import numpy as np

def text_to_char_features(text, sequence_length, step_size):
  """
    Transforms the raw text into training and fitting data.

    x: Train data is a vectorized list of sequences pulled from the text.
    y: fit data is the character following the corresponding sequence in x
    indices_char: dict used to transform back from index to character
  """
  maxlen = sequence_length
  step = step_size
  sequences = []
  next_chars = []
  for i in range(0, len(text) - maxlen, step):
      sequences.append(text[i: i + maxlen])
      next_chars.append(text[i + maxlen])
  print('nb sequences:', len(sequences))
  
  return sequences, next_chars

def create_char_rnn(sequence_length, vocab_size):
  model = Sequential()
  if CHAR_LSTM_LAYERS == 1:
    model.add(CuDNNLSTM(CHAR_LSTM_UNITS, batch_input_shape = (CHAR_BATCH_SIZE, sequence_length, vocab_size), stateful=True))
    model.add(Dropout(CHAR_DROPOUT))
  else:
    for i in range(CHAR_LSTM_LAYERS):
      if i == 0:
        model.add(CuDNNLSTM(CHAR_LSTM_UNITS, batch_input_shape = (CHAR_BATCH_SIZE, sequence_length, vocab_size), return_sequences=True, stateful=True))
      elif i == CHAR_LSTM_LAYERS - 1:
        model.add(CuDNNLSTM(CHAR_LSTM_UNITS, return_sequences=False, stateful=True))
      else:
        model.add(CuDNNLSTM(CHAR_LSTM_UNITS, return_sequences=True, stateful=True))
      model.add(Dropout(CHAR_DROPOUT))
  model.add(Dense(vocab_size, activation = 'softmax'))
  return model

def generator(sequences, next_chars, batch_size):
  i = 0
  while True:
    x = np.zeros((batch_size, CHAR_SEQUENCE_LEN, len(char_to_idx)))
    y = np.zeros((batch_size, len(char_to_idx)))
    for j in range(batch_size):
      sequence = sequences[i]
      for t, char in enumerate(sequence):
        x[j, t, char_to_idx[char]] = 1
      y[j, char_to_idx[next_chars[i]]] = 1
      i += 1
      if i == len(sequences) - 1:
        i = 0
    yield x, y

vocab_size = len(char_to_idx)
print("vocab size:", vocab_size)
sequence_length = CHAR_SEQUENCE_LEN

checkpoint = ModelCheckpoint("models/char_rnn_weights.hdf5",
 save_weights_only=True, monitor='loss', verbose=1,
 save_best_only=False, mode='auto', save_freq='epoch')

optimizer = tf.keras.optimizers.Adam(lr = CHAR_LR)

ModuleNotFoundError: No module named 'tensorflow'

In [0]:
recipe_batch = get_recipes(train_files)
recipe_batch_val = get_recipes(dev_files)
char_sequences, next_chars = text_to_char_features(recipe_batch, CHAR_SEQUENCE_LEN, CHAR_STEP_SIZE)
char_sequences_val, next_chars_val = text_to_char_features(recipe_batch_val, CHAR_SEQUENCE_LEN, CHAR_STEP_SIZE)

nb sequences: 4548834
nb sequences: 1682037


In [0]:
model = create_char_rnn(CHAR_SEQUENCE_LEN, vocab_size)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.summary()
model.fit(generator(char_sequences, next_chars, CHAR_BATCH_SIZE), steps_per_epoch = int(len(char_sequences)/CHAR_BATCH_SIZE) + 1, epochs = CHAR_EPOCHS, callbacks=[checkpoint])

nb sequences: 4667563
nb sequences: 2080212
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_7 (CuDNNLSTM)     (256, 25, 256)            391168    
_________________________________________________________________
dropout_7 (Dropout)          (256, 25, 256)            0         
_________________________________________________________________
cu_dnnlstm_8 (CuDNNLSTM)     (256, 256)                526336    
_________________________________________________________________
dropout_8 (Dropout)          (256, 256)                0         
_________________________________________________________________
dense_3 (Dense)              (256, 124)                31868     
Total params: 949,372
Trainable params: 949,372
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
18232/18233 [============================>.] - ETA: 0s - l

In [0]:
# load model from saved weight file
# so that training can continue between sessions and testing can happen at will
char_rnn = create_char_rnn(CHAR_SEQUENCE_LEN, vocab_size)
char_rnn.load_weights("models/char_rnn_weights.hdf5")
char_rnn.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# evaluate on cross validation set
recipe_batch_val = get_recipes(dev_files)
char_rnn.evaluate(generator(char_sequences_val, next_chars_val, CHAR_BATCH_SIZE), steps = int(len(char_sequences_val)/CHAR_BATCH_SIZE) + 1)

6571/6571 [==============================] - 39s 6ms/step - loss: 2.0760 - accuracy: 0.7037


[2.075989007949829, 0.7036564350128174]

In [0]:
def char_sample(predictions, temperature = 1):
    pred = np.asarray(predictions).astype('float64')
    pred = np.log(pred) / temperature
    exp_pred = np.exp(pred)
    pred = exp_pred / np.sum(exp_pred)
    prob = np.random.multinomial(1, pred, 1)
    return np.argmax(prob)

def char_predict(max_len = 10000, temperature = 0.5):
  index = np.random.randint(len(char_sequences_val))
  sentence = char_sequences_val[index]
  predicted_text = ""
  while '[end]' not in predicted_text and len(predicted_text) < max_len:
    x = np.zeros((CHAR_BATCH_SIZE, CHAR_SEQUENCE_LEN, len(char_to_idx)))
    for t, char in enumerate(sentence):
      x[0, t, char_to_idx[char]] = 1
    pred = char_rnn.predict_on_batch(x)[0]
    prediction_index = char_sample(pred, temperature)
    predicted_letter = idx_to_char[prediction_index]
    predicted_text += predicted_letter
    sentence = sentence[1:] + predicted_letter
  return predicted_text

char_predicted_text = char_predict()
print(char_predicted_text)

gen_file = "generated/char_gentext_{:%Y%m%d_%H%M%S}.txt".format(datetime.utcnow())
file = open(gen_file, "w")
file.write(char_predicted_text)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until











[end]


FileNotFoundError: ignored

# Word-rnn

In [10]:
import pickle
import nltk

def create_tokens(text):
  tokens = nltk.word_tokenize(text)
  return tokens

def write_tokens_to_file(tokens, name):
  with open("tokens.pkl", "wb") as file:
    pickle.dump(tokens, file)

def load_tokens(name):
  with open(name, "rb") as file:
    return pickle.load(file)

def create_word_mapping(tokens):
  words = set(tokens)
  word_to_idx = dict((c, i) for i, c in enumerate(words))
  idx_to_word = dict((i, c) for i, c in enumerate(words))
  return word_to_idx, idx_to_word

def write_word_vocab_to_file(idx_to_word):
  words = [idx_to_word[i] for i in range(len(idx_to_word))]
  with open("words.pkl", "wb") as file:
    pickle.dump(words, file)


def load_word_mapping():
  with open("words.pkl", "rb") as file:
    words = pickle.load(file)

  words_to_idx = dict((c, i) for i, c in enumerate(words))
  idx_to_words = dict((i, c) for i, c in enumerate(words))
  return words_to_idx, idx_to_words

# all_tokens = create_tokens(get_recipes(recipe_files))
# write_tokens_to_file(all_tokens, "alltokens.pk1")
# train_tokens = create_tokens(get_recipes(train_files))
# write_tokens_to_file(all_tokens, "traintokens.pk1")
# val_tokens = create_tokens(get_recipes(dev_files))
# write_tokens_to_file(val_tokens, "valtokens.pk1")

all_tokens = load_tokens("alltokens.pk1")
train_tokens = load_tokens("traintokens.pk1")
val_tokens = load_tokens("valtokens.pk1")

# word_to_idx, idx_to_word = create_word_mapping(all_tokens)
# write_word_vocab_to_file(idx_to_word)

word_to_idx, idx_to_word = load_word_mapping()
print(len(all_tokens), len(train_tokens), len(val_tokens))
print(len(word_to_idx), len(idx_to_word))


7909454 4567616 1655639
16198 16198


In [11]:
WORD_LSTM_LAYERS = 1
WORD_LSTM_UNITS = 256
WORD_SEQUENCE_LEN = 20
WORD_STEP_SIZE = 15
WORD_EPOCHS = 10
WORD_BATCH_SIZE = 32
WORD_DROPOUT = .1
WORD_LR = 0.0005

In [0]:
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, LSTM, Embedding, CuDNNLSTM, Dropout
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import nltk


def text_to_word_features(tokens, sequence_length, step_size):
  # cut the text in semi-redundant sequences of maxlen characters

  maxlen = sequence_length
  step = step_size
  sequences = []
  next_words = []
  for i in range(0, len(tokens) - maxlen, step):
      sequences.append(tokens[i:i+maxlen])
      next_words.append(tokens[i + maxlen])
  print('nb sequences:', len(sequences))

  return sequences, next_words

def create_word_rnn(sequence_length, vocab_size, batch_size):
  model = Sequential()
  if WORD_LSTM_LAYERS == 1:
    model.add(CuDNNLSTM(WORD_LSTM_UNITS, batch_input_shape = (WORD_BATCH_SIZE, sequence_length, vocab_size), stateful=True))
    model.add(Dropout(WORD_DROPOUT))
  else:
    for i in range(WORD_LSTM_LAYERS):
      if i == 0:
        model.add(CuDNNLSTM(WORD_LSTM_UNITS, batch_input_shape = (WORD_BATCH_SIZE, sequence_length, vocab_size), return_sequences=True, stateful=True))
      elif i == WORD_LSTM_LAYERS - 1:
        model.add(CuDNNLSTM(WORD_LSTM_UNITS, return_sequences=False, stateful=True))
      else:
        model.add(CuDNNLSTM(WORD_LSTM_UNITS, return_sequences=True, stateful=True))
      model.add(Dropout(WORD_DROPOUT))
  model.add(Dense(vocab_size, activation = 'softmax'))
  return model

def generator(sequences, next_words, batch_size):
  i = 0
  while True:
    x = np.zeros((batch_size, WORD_SEQUENCE_LEN, len(word_to_idx)))
    y = np.zeros((batch_size, len(word_to_idx)))
    for j in range(batch_size):
      sequence = sequences[i]
      for t, word in enumerate(sequence):
        x[j, t, word_to_idx[word]] = 1
      y[j, word_to_idx[next_words[i]]] = 1
      i += 1
      if i == len(sequences) - 1:
        i = 0
    yield x, y

word_vocab_size = len(word_to_idx)
word_sequence_length = WORD_SEQUENCE_LEN

word_checkpoint = ModelCheckpoint("models/word_rnn_weights.hdf5",
 save_weights_only=True, monitor='loss', verbose=1,
 save_best_only=True, mode='auto', save_freq='epoch')


optimizer = tf.keras.optimizers.Adam(lr = WORD_LR)

In [0]:
word_sequences, next_words = text_to_word_features(train_tokens, WORD_SEQUENCE_LEN, WORD_STEP_SIZE)
word_sequences_val, next_words_val = text_to_word_features(val_tokens, WORD_SEQUENCE_LEN, WORD_STEP_SIZE)

nb sequences: 416646
nb sequences: 96108


In [0]:
model = create_word_rnn(word_sequence_length, word_vocab_size, WORD_BATCH_SIZE)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.summary()
model.fit(generator(word_sequences, next_words, WORD_BATCH_SIZE), steps_per_epoch = int(len(sequences)/WORD_BATCH_SIZE) + 1, epochs = WORD_EPOCHS, callbacks=[word_checkpoint])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm (CuDNNLSTM)       (32, 256)                 68114432  
_________________________________________________________________
dropout (Dropout)            (32, 256)                 0         
_________________________________________________________________
dense (Dense)                (32, 66260)               17028820  
Total params: 85,143,252
Trainable params: 85,143,252
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
13020/13021 [============================>.] - ETA: 0s - loss: 5.3691 - acc: 0.1760
Epoch 00001: loss improved from inf to 5.36922, saving model to content/drive/My Drive/CookingWithBert/models/word_rnn_weights.hdf5
13021/13021 [================

In [0]:
word_rnn = create_word_rnn(word_sequence_length, word_vocab_size, WORD_BATCH_SIZE)
word_rnn.load_weights("models/word_rnn_weights.hdf5")
word_rnn.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
# word_rnn.evaluate(generator(word_sequences_val, next_words_val, WORD_BATCH_SIZE), steps = int(len(sequences)/WORD_BATCH_SIZE) + 1)
word_rnn.evaluate(generator(word_sequences_val, next_words_val, WORD_BATCH_SIZE), steps = 1000)

1000/1000 [==============================] - 136s 136ms/step - loss: 4.1885 - accuracy: 0.3184


[4.188460350036621, 0.31840625405311584]

In [0]:
def word_sample(predictions, temperature = 1):
    pred = np.asarray(predictions).astype('float64')
    pred = np.log(pred) / temperature
    exp_pred = np.exp(pred)
    pred = exp_pred / np.sum(exp_pred)
    prob = np.random.multinomial(1, pred, 1)
    return np.argmax(prob)

def word_prediction(max_len = 1000, temp = 0.5):
  index = np.random.randint(len(word_sequences_val))
  sentence = word_sequences_val[index]
  predicted_text = []
  while (len(predicted_text) == 0 or predicted_text[-1] is not ']') and len(predicted_text) < max_len:
    x = np.zeros((WORD_BATCH_SIZE, WORD_SEQUENCE_LEN, len(word_to_idx)))
    for t, word in enumerate(sentence):
      x[0, t, word_to_idx[word]] = 1
    pred = word_rnn.predict(x, verbose = 0)[0]
    prediction_index = sample(pred, temp)
    predicted_word = idx_to_word[prediction_index]
    predicted_text += [predicted_word]
    sentence = sentence[1:] + [predicted_word]
  return ' '.join(predicted_text)

word_predicted_text = word_prediction()
print(word_predicted_text)

gen_file = "generated/word_gentext_{:%Y%m%d_%H%M%S}.txt".format(datetime.utcnow())
file = open(gen_file, "w")
file.write(word_predicted_text)


: butter : rub 's dressing categories : fish yield : 4 servings 3 lb boneless chicken breasts 1 lb butter 1 cn ( 7 oz ) tomato paste 6 oz can tomato sauce 2 tb chili powder 1 tb wine vinegar 1 tb honey 1 ts dry mustard 1 tb lemon juice 1/2 ts salt 1/2 ts pepper 1/4 ts dried tarragon 2 ts dried oregano 1/4 ts dried rosemary 1/4 ts dried basil 1/4 ts dried oregano 1/4 ts dried thyme 1/4 ts dried rosemary 1/2 ts white pepper 1/4 ts salt 1/4 ts pepper 1 c buttermilk , mixed with 1/2 c water 1/4 c sugar 1/4 c butter flavor melted butter 1 ts vanilla 2 tb butter 1 ts vanilla 2 ts baking powder 1 ts baking soda 1 ts baking soda 1 c milk 2 tb butter 2 tb butter 2 c flour 1 ts baking soda 1 ts baking soda 1 ts baking soda 1 ts vanilla 1 c chopped walnuts 1 1/2 c chopped pecans ( or less ) 1 ts vanilla extract [ end ]


FileNotFoundError: ignored

# GPT-2

In [0]:
%tensorflow_version 1.x
# !pip uninstall tensorflow
# !pip install tensorflow==1.15
!pip3 install gpt-2-simple

import tensorflow as tf
import gpt_2_simple as gpt2
import os


TensorFlow 1.x selected.
  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.7.1-cp36-none-any.whl size=23581 sha256=c32c4a6aa052e877ae14fdf3ad7bdddd1c148b0397633729c9a80a27a392b53f
  Stored in directory: /root/.cache/pip/wheels/0c/f8/23/b53ce437504597edff76bf9c3b8de08ad716f74f6c6baaa91a
Successfully built gpt-2-simple
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
# load from google drive and run trainng regemine

model_name = "124M"

if not os.path.isdir(os.path.join("models", model_name)):
	print(f"Downloading {model_name} model...")
	gpt2.download_gpt2(model_name=model_name)   # model is saved into current directory under /models/124M/


run = "run1"
gpt2.copy_checkpoint_from_gdrive(run_name=run)

# in case restarting training in the middle due to a disconnect.
start_index = 18

for i in range(start_index, len(train_files)):
  print("Made it to %d file index." % (i))
  recipe_file = train_files[i]

  tf.reset_default_graph()
  sess = gpt2.start_tf_sess()
  gpt2.finetune(sess,
                dataset=recipe_file,
                model_name='124M',
                steps=500,
                restore_from='latest',
                run_name=run,
                print_every=25,
                sample_every=500,
                save_every=250
                )
  gpt2.copy_checkpoint_to_gdrive(run_name=run)


Fetching checkpoint: 1.05Mit [00:00, 538Mit/s]                                                      


Fetching encoder.json: 1.05Mit [00:00, 75.3Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 587Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:03, 151Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 392Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 157Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 149Mit/s]                                                       


Made it to 18 file index.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint checkpoint/run1/model-20000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-20000


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:02<00:00,  2.24s/it]


dataset has 378721 tokens
Training...
Saving checkpoint/run1/model-20000
======== SAMPLE 1 ========
.  Put bacon strips in a flameproof casserole and
  pour in half-and-half.  Mix flour, sugar and salt into saucepan, then add
  water and bring to the boil.  Add salt and baking powder and mix well. Put
  turkey in casserole, cover, and marinate overnight.  (I have found that
  egg dishes have better than that, but I like my turkey good.)
  
  Preheat oven to 325 deg f.  Unmednately melt butter in a pan and put in
  the oven, turn off the oven and leave turkey in the casserole overnight.
  Unmold turkey and marinate with sauce overnight, or put turkey on a
  plate and top with sauce.  Sprinkle with grated cheese, or serve with
  butter. From: Stephanie Da Silva
 
[END] 
[END] 
      Title: GOLDEN TURKEY BREAST PARMIGIANA
 Categories: Poultry
      Yield: 4 Servings
 
      2 lb Whole turkey breast
      1 lg Turkey breast
    1/2 c  Melted butter
      2 tb Flour
      1 ts Lemon pepper


In [0]:
from datetime import datetime

# To reload from google drive and generate

run = "run1"
gpt2.copy_checkpoint_from_gdrive(run_name=run)

tf.reset_default_graph()
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=run)

gen_file = "/content/drive/My Drive/CookingWithBert/generated/gpt2_gentext_{:%Y%m%d_%H%M%S}.txt".format(datetime.utcnow())
gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

Loading checkpoint checkpoint/run1/model-20500
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-20500
